In [1]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.5 in /home/hadil/.local/lib/python3.8/site-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision==0.6 in /home/hadil/.local/lib/python3.8/site-packages (0.6.0+cu101)
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-mp38y5cl
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-mp38y5cl
  Created wheel for pycocotools: filename=pycocotools-2.0-cp38-cp38-linux_x86_64.whl size=418980 sha256=a162315e344aad70b57e203f3d8445a8bcbf0214aeab9642ef067ec2ccfd0af1
  Stored in directory: /tmp/pip-ephem-wheel-cache-yw3_iasf/wheels/56/da/49/cb71a7c450b59588934077f431100c05fbde50646ee84a8d40
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0
1.5.0+cu101 False
gcc (Ubun

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle

from PIL import Image
import numpy as np
import pickle

from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm

In [4]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html


Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html


In [3]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import os
import numpy as np
import cv2
import random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [4]:
#liée la BD
import psycopg2
DB_Host='localhost'
DB_Name='chatbot'
DB_User='postgres'
DB_pass='hadil24'
connection = psycopg2.connect(dbname=DB_Name,user=DB_User,password=DB_pass,host=DB_Host)
cursor=connection.cursor()

In [5]:
from detectron2.data.datasets import register_coco_instances
#register_coco_instances("deepfashion_train", {}, "/content/DeepFashion2/deepfashion2_train.json", "/content/DeepFashion2/train/image")
register_coco_instances("deepfashion_val", {}, "/home/hadil/final/deepfashion2_validation.json", "/home/hadil/final/validation/image")

In [6]:
cfg = get_cfg() #la configuration par défaut de détéctron
cfg.MODEL.DEVICE='cpu'
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")) #builtin config file.
cfg.DATASETS.TRAIN = ("deepfashion_val",)
cfg.DATASETS.TEST = ()

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo 
                                                                                                  #pre-trained weights
                                                                                                  #URL to the model trained using the given config

"""_BASE_: "../Base-RCNN-FPN.yaml"
MODEL:
  WEIGHTS: "detectron2://ImageNetPretrained/MSRA/R-101.pkl"
  MASK_ON: False
  RESNETS:
    DEPTH: 101
SOLVER:
  STEPS: (210000, 250000)
  MAX_ITER: 270000
  """                                                                                                  

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1500  #nombre d'itération
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13 #nombre de classe

cfg.TEST.EVAL_PERIOD = 500

In [7]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True) # create directory output_dir
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False) 

[06/03 05:19:41 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (14, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (14,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (52, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (52,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias

In [8]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/home/hadil/final/model_final.pth")  #trained model path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.55   # set the testing threshold for this model
                                               # seuil utilisé pour filtrer low-scored bounding boxes predicted by the Fast R-CNN during the test
cfg.DATASETS.TEST = ("deepfashion_val", )
predictor = DefaultPredictor(cfg)

In [8]:
im = cv2.imread("/home/hadil/final/anchor/sss.jpg")
outputs = predictor(im) #output is a dictionary
#Metadata is a key-value used to interpret what’s in the dataset( names of classes)..
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))  #moving outputs["instances"] to cpu then apply draw_instance_predictions
cv2.imshow('imgg',v.get_image()[:, :, ::-1])
cv2.waitKey(0)
import uuid #helps in generating random objects of 128 bits as ids

boxes = {}
for coordinates in outputs["instances"].to("cpu").pred_boxes:
  coordinates_array = []
  for k in coordinates:
    coordinates_array.append(int(k))
  
  boxes[uuid.uuid4().hex[:].upper()] = coordinates_array
for k,v in boxes.items():
  crop_img = im[v[1]:v[3], v[0]:v[2], :]
  cv2.imshow('imgg_class.jpg',crop_img)
  cv2.imwrite('img_class.jpg', crop_img)
  cv2.waitKey(0)

AttributeError: 'NoneType' object has no attribute 'shape'

In [9]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

model = tf.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

2022-06-03 05:20:03.041209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
lr = 0.1
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [10]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [61]:
filenames=[]

print("\nDisplaying Entire Table product ")
cursor.execute("SELECT * FROM attachment;")
output=cursor.fetchall()
for item in output:
    print(item[1])
    filenames.append(item[1])
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224)) #charger une img
    img_array = image.img_to_array(img) #convertir en array
    expanded_img_array = np.expand_dims(img_array, axis=0) #(1,224,224,3)
    preprocessed_img = preprocess_input(expanded_img_array) #The images are converted from RGB to BGR, then each color channel is zero-centered with respect to the ImageNet dataset
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result
feature_list = []

for file in tqdm(filenames):
    feature_list.append(extract_features(file,model))


Displaying Entire Table product 
/home/hadil/final/anchor/01.jpeg
/home/hadil/final/anchor/02.jpg
/home/hadil/final/anchor/03.jpg
/home/hadil/final/anchor/04.jpg
/home/hadil/final/anchor/05.jpg
/home/hadil/final/anchor/06.jpeg
/home/hadil/final/anchor/07.jpg
/home/hadil/final/anchor/08.jpg
/home/hadil/final/anchor/004.jpg
/home/hadil/final/anchor/0044.jpg
/home/hadil/final/anchor/09.jpeg
/home/hadil/final/anchor/20.jpeg


  0%|          | 0/12 [00:00<?, ?it/s]

1/1 [==============================] - 0s 133ms/step


  8%|▊         | 1/12 [00:00<00:02,  5.43it/s]

1/1 [==============================] - 0s 98ms/step


...

 83%|████████▎ | 10/12 [00:01<00:00,  7.38it/s]

1/1 [==============================] - 0s 88ms/step


 92%|█████████▏| 11/12 [00:01<00:00,  7.47it/s]

1/1 [==============================] - 0s 91ms/step


100%|██████████| 12/12 [00:01<00:00,  7.15it/s]


In [36]:
"""
print(os.listdir('/home/hadil/final/anchor'))
filenames = []

for file in os.listdir('/home/hadil/final/anchor'):
    filenames.append(os.path.join('/home/hadil/final/anchor',file))
print(filenames[0:5])
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224)) #charger une img
    img_array = image.img_to_array(img) #convertir en array
    expanded_img_array = np.expand_dims(img_array, axis=0) #(1,224,224,3)
    preprocessed_img = preprocess_input(expanded_img_array) #The images are converted from RGB to BGR, then each color channel is zero-centered with respect to the ImageNet dataset
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result
feature_list = []

for file in tqdm(filenames):
    feature_list.append(extract_features(file,model))
    """

"\nprint(os.listdir('/home/hadil/final/anchor'))\nfilenames = []\n\nfor file in os.listdir('/home/hadil/final/anchor'):\n    filenames.append(os.path.join('/home/hadil/final/anchor',file))\nprint(filenames[0:5])\ndef extract_features(img_path,model):\n    img = image.load_img(img_path,target_size=(224,224)) #charger une img\n    img_array = image.img_to_array(img) #convertir en array\n    expanded_img_array = np.expand_dims(img_array, axis=0) #(1,224,224,3)\n    preprocessed_img = preprocess_input(expanded_img_array) #The images are converted from RGB to BGR, then each color channel is zero-centered with respect to the ImageNet dataset\n    result = model.predict(preprocessed_img).flatten()\n    normalized_result = result / norm(result)\n\n    return normalized_result\nfeature_list = []\n\nfor file in tqdm(filenames):\n    feature_list.append(extract_features(file,model))\n    "

In [69]:
#print(os.listdir('/home/hadil/final/anchor'))
#filenames = []

#for file in os.listdir('/home/hadil/final/anchor'):
 #   filenames.append(os.path.join('/home/hadil/final/anchor',file))
#print(filenames[0:5])

['IMG_20220408_085855.jpg', 'pull_monde.jpg', 'hmgoepprod (13).jpg', 'Copie de hmgoepprod (31).jpg', 'hmgoepprod (1).jpg', 'Copie de blanc2.jpeg', 'IMG_20220408_085912.jpg', 'IMG_20220408_085731.jpg', 'Copie de bleu clair1.jpeg', 'Copie de berbey1.jpeg', 'Copie de hmgoepprod.jpg', 'IMG_20220408_085743.jpg', 'hmgoepprod (1).jpeg', 'hmgoepprod (17).jpg', 'IMG_20220408_085903.jpg', 'Copie de beige2.jpeg', 'Copie de blanc twil1.jpeg_', 'Copie de hmgoepprod (32).jpg', 'hmgoepprod (31).jpg', 'short.jpeg', 'hmgoepprod (29).jpg', 'hmgoepprod (12).jpg', 'Copie de hmgoepprod (23).jpg', 'Copie de rose1.jpeg', 'IMG_20220408_085827.jpg', 'hmgoepprod.jpg', 'Copie de orange.jpeg', 'hmgoepprod (4).jpg', 'IMG_20220408_085818.jpg', 'Copie de hmgoepprod (24).jpg', 'hmgoepprod (3).jpg', 'hmgoepprod (6).jpg', 'Copie de beige1.jpeg', 'IMG_20220408_085936.jpg', 'orange1.jpeg', 'Copie de berbery.jpeg', 'hmgoepprod (19).jpg', 'hmgoepprod (7).jpg', 'Copie de hmgoepprod (30).jpg', 'hmgoepprod (8).jpg', 'Copie de

In [62]:
pickle.dump(feature_list,open('embedding.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))
feature_list=pickle.load(open('embedding.pkl','rb')) #load features list
filenames=pickle.load(open('filenames.pkl','rb')) #load image's paths list


In [63]:
img = image.load_img('/home/hadil/final/anchor/004.jpg',target_size=(224,224)) #charger une img
img_array = image.img_to_array(img) #convertir en array
expanded_img_array = np.expand_dims(img_array, axis=0) #(1,224,224,3)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)
neighbors= NearestNeighbors(n_neighbors=3,algorithm='brute', metric='euclidean') #Number of neighbors to use=5, ‘brute’ will use a brute-force search, euclidean: distance entre 2 points
neighbors.fit(feature_list)
distances,indices= neighbors.kneighbors([normalized_result])
print(indices)
print(distances)
for file in indices[0]:
  print(filenames[file])
for file in indices[0]:
  similar=cv2.imread(filenames[file])
  cv2.imshow('sim',similar)
  cv2.waitKey(0)
 

1/1 [==============================] - 0s 115ms/step
[[ 8  3 11]]
[[4.4703484e-08 6.4920336e-01 6.5273458e-01]]
/home/hadil/final/anchor/004.jpg
/home/hadil/final/anchor/04.jpg
/home/hadil/final/anchor/20.jpeg


In [19]:
im = cv2.imread("/home/hadil/final/anchor/hmgoepprod (7).jpg")
outputs = predictor(im) #output is a dictionary
#Metadata is a key-value used to interpret what’s in the dataset( names of classes)..
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))  #moving outputs["instances"] to cpu then apply draw_instance_predictions
cv2.imshow('imgg',v.get_image()[:, :, ::-1])
cv2.waitKey(0)

-1

In [30]:
im = cv2.imread("/home/hadil/final/anchor/hmgoepprod (7).jpg")
outputs = predictor(im) #output is a dictionary
#Metadata is a key-value used to interpret what’s in the dataset( names of classes)..
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))  #moving outputs["instances"] to cpu then apply draw_instance_predictions
cv2.imshow('imgg',v.get_image()[:, :, ::-1])
cv2.waitKey(0)
import uuid #helps in generating random objects of 128 bits as ids

boxes = {}
for coordinates in outputs["instances"].to("cpu").pred_boxes:
  coordinates_array = []
  for k in coordinates:
    coordinates_array.append(int(k))


  if(len(coordinates_array)==0):
    img = image.load_img('/home/hadil/final/anchor/hmgoepprod (7).jpg',target_size=(224,224)) #charger une img
    img_array = image.img_to_array(img) #convertir en array
    expanded_img_array = np.expand_dims(img_array, axis=0) #(1,224,224,3)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)
    neighbors= NearestNeighbors(n_neighbors=3,algorithm='brute', metric='euclidean') #Number of neighbors to use=5, ‘brute’ will use a brute-force search, euclidean: distance entre 2 points
    neighbors.fit(feature_list)
    distances,indices= neighbors.kneighbors([normalized_result])
    print(indices)
    print(distances)
    for file in indices[0]:
      print(filenames[file])
    for file in indices[0]:
      similar=cv2.imread(filenames[file])
      cv2.imshow('sim',similar)
      cv2.waitKey(0)
      
  else: 
      boxes[uuid.uuid4().hex[:].upper()] = coordinates_array
      for k,v in boxes.items():
         crop_img = im[v[1]:v[3], v[0]:v[2], :]
  
         cv2.imshow('part1.jpg',crop_img)
         cv2.imwrite('part1.jpg', crop_img)
         cv2.waitKey(0) 















  
   



In [68]:
# toutes les instances détéctées
im = cv2.imread("/home/hadil/final/anchor/04.jpg")
outputs = predictor(im) #output is a dictionary
#Metadata is a key-value used to interpret what’s in the dataset( names of classes)..
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))  #moving outputs["instances"] to cpu then apply draw_instance_predictions
cv2.imshow('imgg',v.get_image()[:, :, ::-1])
cv2.waitKey(0)

import uuid #helps in generating random objects of 128 bits as ids

boxes = {} 
for coordinates in outputs["instances"].to("cpu").pred_boxes:
  coordinates_array = [] #les coordonnées de chaque instance
  for k in coordinates:
    coordinates_array.append(int(k))
  boxes[uuid.uuid4().hex[:].upper()] = coordinates_array  #les instance lkol fi dict
#crop predicted instances
for k,v in boxes.items():
  crop_img = im[v[1]:v[3], v[0]:v[2], :]
  
  cv2.imshow('part1.jpg',crop_img)
  cv2.imwrite('part1.jpg', crop_img)
  cv2.waitKey(0)  


In [66]:
######## Instance 0
im = cv2.imread("/home/hadil/final/anchor/04.jpg")
outputs = predictor(im) #output is a dictionary contient les instances 
#detect articles
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))  #moving outputs["instances"] to cpu then apply draw_instance_predictions
cv2.imshow('imgg',v.get_image()[:, :, ::-1])
cv2.waitKey(0)


import uuid #helps in generating random objects of 128 bits as ids

boxes = {}
for coordinates in outputs["instances"][0].to("cpu").pred_boxes:
  coordinates_array = []
  for k in coordinates:
    coordinates_array.append(int(k))
  
  boxes[uuid.uuid4().hex[:].upper()] = coordinates_array
for k,v in boxes.items():
  crop_img = im[v[1]:v[3], v[0]:v[2], :]
  
  cv2.imshow('part1.jpg',crop_img)
  cv2.imwrite('part1.jpg', crop_img)
  cv2.waitKey(0)
  



img = image.load_img('part1.jpg',target_size=(224,224)) #charger une img
img_array = image.img_to_array(img) #convertir en array
expanded_img_array = np.expand_dims(img_array, axis=0) #(1,224,224,3)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)
neighbors= NearestNeighbors(n_neighbors=3,algorithm='brute', metric='euclidean') #Number of neighbors to use=5, ‘brute’ will use a brute-force search, euclidean: distance entre 2 points
neighbors.fit(feature_list)
distances,indices= neighbors.kneighbors([normalized_result])
print(indices)
print(distances)
for file in indices[0]:
  print(filenames[file])
for file in indices[0]:
  similar=cv2.imread(filenames[file])
  cv2.imshow('sim',similar)
  cv2.waitKey(0)
 

1/1 [==============================] - 0s 137ms/step
[[ 3  8 11]]
[[0.646863   0.71235025 0.7554165 ]]
/home/hadil/final/anchor/04.jpg
/home/hadil/final/anchor/004.jpg
/home/hadil/final/anchor/20.jpeg


In [57]:
#instance1
import uuid #helps in generating random objects of 128 bits as ids

boxes = {}
for coordinates in outputs["instances"][1].to("cpu").pred_boxes:
  coordinates_array = []
  for k in coordinates:
    coordinates_array.append(int(k))
  
  boxes[uuid.uuid4().hex[:].upper()] = coordinates_array
for k,v in boxes.items():
  crop_img = im[v[1]:v[3], v[0]:v[2], :]
  
  cv2.imshow('part2.jpg',crop_img)
  cv2.imwrite('part2.jpg', crop_img)
  cv2.waitKey(0)
  



img = image.load_img('part2.jpg',target_size=(224,224)) #charger une img
img_array = image.img_to_array(img) #convertir en array
expanded_img_array = np.expand_dims(img_array, axis=0) #(1,224,224,3)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)
neighbors= NearestNeighbors(n_neighbors=3,algorithm='brute', metric='euclidean') #Number of neighbors to use=5, ‘brute’ will use a brute-force search, euclidean: distance entre 2 points
neighbors.fit(feature_list)
distances,indices= neighbors.kneighbors([normalized_result])
print(indices)
print(distances)
for file in indices[0]:
  print(filenames[file])
for file in indices[0]:
  similar=cv2.imread(filenames[file])
  cv2.imshow('sim',similar)
  cv2.waitKey(0)

1/1 [==============================] - 0s 97ms/step
[[ 4  6 10]]
[[0.5405096  0.6975931  0.69925404]]
/home/hadil/final/anchor/05.jpg
/home/hadil/final/anchor/07.jpg
/home/hadil/final/anchor/09.jpeg
